In [1]:
import pandas as pd
import numpy as np

from IPython.display import Markdown, display
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster, datasets
from sklearn.cluster import KMeans
import random
import pickle


In [2]:
#Importing required file


In [3]:
films = pd.read_csv('films_w_awards_cluster.csv', index_col =False)


/var/folders/0r/l856hq9x61zf5p8g8vxjs_dh0000gn/T/ipykernel_3320/1553699301.py:1: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34,35,36,37,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  films = pd.read_csv('films_w_awards_cluster.csv', index_col =False)


In [4]:
# Ask for number of movie names to enter
while True:
    try:
        num_movies = int(input("How many people are you? (Enter a number between 1 and 5): "))
        # validating number of inputs
        if num_movies < 1 or num_movies > 5:
            print("Please enter a number between 1 and 5.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a number between 1 and 5.")


# Create an empty list to store movie names
movie_names = []

# Loop to get movie names based on the number entered
for i in range(num_movies):
    while True:
        movie_name = input("Please enter a movie that you love (original name) #{}: ".format(i+1))
        search_result = films[films['movie_name'].str.lower().str.contains(movie_name.lower(), case=False, na=False)]
        if len(search_result) == 0:
            print("Ups, it seems that we don't know that movie... Please try again.")
        else:
            movie_names.append(movie_name.lower())
            break

# Search for the movie names in the 'films' dataframe
movie_recommendations = []


for user_input in movie_names:
    search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

    if len(search_result) > 0:
        # If multiple options found, ask user to choose one
        if len(search_result) > 1:
            print("We have found different movies for that name! Please choose one from the list:")
            for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                print("{}. {} ({})".format(i, movie[1]['movie_name'], movie[1]['year_film']))

            # Continuously ask for input until a valid choice is provided
            while True:
                try:
                    chosen_movie_index = int(input("Enter the number corresponding to the movie you are referring to: "))
                    if chosen_movie_index > 0 and chosen_movie_index <= len(search_result):
                        chosen_movie_result = search_result.iloc[chosen_movie_index-1]
                        chosen_movie_name = chosen_movie_result['movie_name']
                        print("")
                        print("You have chosen: {}".format(chosen_movie_name))
                        print("")
                        break
                    else:
                        print("")
                        print("Invalid choice. Please select a number from the list.")
                        print("")
                        
                except ValueError:
                    print("")
                    print("Invalid choice. Please select a number from the list.")
                    print("")
        else:
            chosen_movie_result = search_result.iloc[0]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("")
            print("You have chosen: {}".format(chosen_movie_name))
            print("")
        # Get the cluster of the chosen movie
        cluster = chosen_movie_result['cluster']
        # Get the year_film of the chosen movie
        year_film = chosen_movie_result['year_film']

        # Convert year_film to integer
        year_film = int(year_film)

        print("")
        print("Searched movie:")
        print(chosen_movie_result[['movie_name', 'year_film', 'director']])
        print("")
        
        # Filter films dataframe by cluster
        films_in_same_cluster = films[films['cluster'] == cluster]

        if len(films_in_same_cluster) > 0:
            # Filter films in same cluster with close year_film
            films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
            if len(films_close_year) > 0:
                # Remove chosen movie from the recommendations
                films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                # Sort films by rating in descending order
                films_close_year = films_close_year.sort_values(by='rating', ascending=False)
                # Select the 10 highest rated films
                top_10_films = films_close_year.head(10)
                # Print a random film from the same cluster with close year_film
                random_film =  top_10_films.sample(n=1)
            else:
                # Print a random film from the same cluster
                random_film =  top_10_films.sample(n=1)
        else:
            # Print a random film from the entire dataframe
            random_film =  top_10_films.sample(n=1)

        # Extract required columns
        output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes','description','osc_nominated_num', 'osc_won_num', 'cannes_won_num', 'movie_id']].copy()
        # Convert movie_id to string and concatenate with IMDb URL
        output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

        # Append the recommendation to the list of movie recommendations
        movie_recommendations.append(output)

# Display the movie recommendations
if len(movie_recommendations) > 0:
    print("Movie Recommendations:")
    for i, recommendation in enumerate(movie_recommendations, start=1):
        print("{}. Movie Name: {}".format(i, recommendation['movie_name'].values[0]))
        print("   Year: {}".format(recommendation['year_film'].values[0]))
        print("   Director: {}".format(recommendation['director'].values[0]))
        print("   Runtime: {} mins".format(recommendation['runtime'].values[0]))
        print("   Rating: {}".format(recommendation['rating'].values[0]))
        print("   Votes: {}".format(recommendation['votes'].values[0]))
        print("   IMDb URL: {}".format(recommendation['imdb_url'].values[0]))
        print("   Description: {}".format(recommendation['description'].values[0]))
        print("   Oscars Nominations: {}".format(recommendation['osc_nominated_num'].values[0]))
        print("   Oscars Won: {}".format(recommendation['osc_won_num'].values[0]))
        print("   Cannes Won: {}".format(recommendation['cannes_won_num'].values[0]))
        print("")

else:
    print("No movie recommendations found.")

How many people are you? (Enter a number between 1 and 5): 1
Please enter a movie that you love (original name) #1: titanic
We have found different movies for that name! Please choose one from the list:
1. Titanic (1953)
2. Raise the Titanic (1980)
3. The Chambermaid on the Titanic (1997)
4. La leggenda del Titanic (1999)
5. Heroes of the Titanic (2000)
6. Titanic: The Legend Goes On... (2000)
7. In Search of the Titanic (2004)
8. The Ten Lives of Titanic the Cat (2007)
9. Rifftrax: Titanic (2009)
10. Up there Titanic (2012)
11. After the Titanic (2012)
12. Metatitanic (2018)
13. Titanic Ocean (2021)
14. The Blind Man Who Did Not Want to See Titanic (2021)
15. Titanic: Kadhalum Kavunthu Pogum (2022)
16. Titanic 666 (2022)
Enter the number corresponding to the movie you are referring to: 17

Invalid choice. Please select a number from the list.

Enter the number corresponding to the movie you are referring to: 13

You have chosen: Titanic Ocean


Searched movie:
movie_name            Ti